## Initial Setup

Below are some imports and function definitions required for doing the computations.

In [1]:
import pandas as pd
import time
from multiprocessing import Pool


In [2]:
A.<x> = PolynomialRing(CC)
R.<x> = PolynomialRing(QQ)

In [3]:
def boxedPolys(d, b, bound = -1):
    n = d / 2
    bounds = [b * 2 + 1 for i in range(n)]
    # print(bounds)
    
    possibleCoef = xmrange(bounds)
    polys = []
    counter = 0
    
    for coefs in possibleCoef:
        if(bound != -1 and counter == bound):
            break;
        counter += 1

        coefs = [1] + [coef - b for coef in coefs]
        coefs.extend(coefs[:-1][::-1])
        poly = R(coefs)
        
        if not poly.is_irreducible():
            # print(coefs)
            continue
            
        # The section below is for if you want to filter out cyclotomic
        # polynomials and polynomials of high Mahler measure. You need to add
        # an extra parameter, M, to the function then
#         mahler = mahlerMeasure(poly)
        
#         # print(coefs)
        
#         if mahler > M or mahler == 1:
#             # print(mahler)
#             # print("high")
#             continue
        
        polys.append(poly)
    return polys
def getPolyIfIrred(coefs):
    coefs = [1] + [coef - b for coef in coefs]
    coefs.extend(coefs[:-1][::-1])
    poly = R(coefs)

    if not poly.is_irreducible():
        # print(coefs)
        return -1
    return poly
        
def boxedPolysParallel(d, b):
    n = d / 2
    bounds = [b * 2 + 1 for i in range(n)]
    # print(bounds)
    
    possibleCoef = xmrange(bounds)
    with Pool() as pool: 
        polys = pool.map(getPolyIfIrred, possibleCoef)
    polys = [f for f in polys if f != -1]
    
    return polys
def mahlerMeasure(poly):
    tempPoly = A(poly)
    rootList = list(tempPoly.roots())
    measure = 1
    
    for s in rootList:
        s_2 = sqrt(norm(s[0]))
        measure = measure * max(1, s_2)

    return measure

In [4]:
# filters out the reducibles from the list of polynomial objects
def filterOutReducibles(polys):
    newPolys = []
    for poly in polys:
        if poly.is_irreducible():
            newPolys.append(poly)
    return newPolys


# computes the galois group for the given polynomial.
def getGalois(poly):
    # make sure the polynomial is not reducible before
    # computing its Galois group
    # assert poly.is_irreducible()
    K.<a> = NumberField(poly)
    G = K.galois_group()
    return G
    # return G.transitive_label()

def convertToPoly(coefs):
    # Define a polynomial ring for our polynomials and convert coefs to poly using this ring
    R = PolynomialRing(QQ, 'x')
    return R(coefs)


# function to get the roots and Mahler measure

#Returns roots w norm greater than 1
def bigRoots(poly): 
    S.<x> = PolynomialRing(CC)
    tempPoly = S(poly)
    rootList = tempPoly.roots()
    bigRootList = []
    for r in rootList:
        if sqrt(norm(r[0])) > 1:
            bigRootList.append(r)

    return len(bigRootList)

def realRoots(poly): 
    L.<b> = PolynomialRing(RR)
    tempPoly = L(poly)
    rootList = list(tempPoly.roots())
    realRootList = []
    for r in rootList:
        if r[0] == conjugate(r[0]):
            realRootList.append(r)

    return len(realRootList)

def mahlerMeasure(poly):
    S.<x> = PolynomialRing(CC)
    tempPoly = S(poly)
    rootList = list(tempPoly.roots())
    measure = 1
    
    # print(rootList)
    
    # question: are we dealing with multiplicity correctly?
    for s in rootList:
        s_2 = sqrt(norm(s[0]))
        measure = measure * max(1, s_2)

    return measure

#Returns trace polynomial 
def tracePoly(poly):
    K.<a> = NumberField(poly)
    return (a + 1/a).minpoly()

# one goal: plot the number of polynomials with Mahler Measure under a certain bound
# as you increase the degree

# things we want to collect:
# ((f) poly, roots of f outside unit circle, # of real roots, (g) trace poly, G_f, G_g, Mahler, disc)
# collect data into a csv (so we can plot it)

def getStats(poly):
    # poly, degree, Mahler measure, roots outside unit circle, # real roots, trace poly, galois of poly, galois of trace, discriminant, 
    trace = tracePoly(poly)
#     polyGalois = getGalois(poly)
#     polyGaloisLabel = polyGalois.transitive_label()

    traceGalois = getGalois(trace)
    traceGaloisLabel = traceGalois.transitive_label()
    returnArr = [poly 
            , poly.degree()
            , mahlerMeasure(poly) 
            , bigRoots(poly) # roots outside unit circle
            , realRoots(poly) 
            , trace
#             , polyGaloisLabel
            , traceGaloisLabel
            , disc(poly) # discriminant
#             , polyGalois.order()
            , traceGalois.order()
#             , polyGalois.order() / traceGalois.order()
          ] 
#     print("done")
    return returnArr

In [10]:
headers = ["polynomial", "degree", "Mahler measure", "roots outside unit circle", 
           "number of real roots", "trace poly", "Galois of poly", "Galois of trace", 
           "discriminant", "galois order", "trace galois order", "galois ratio"]
headers2 = ["polynomial", "degree", "Mahler measure", "roots outside unit circle", 
           "number of real roots", "trace poly", "Galois of trace", 
           "discriminant", "trace galois order"]

## Data Collection
Set your value of d (degree) and b (bound for coefficients) here. It will also track and print out the time taken to complete all computations.

Feel free to combine the two functions into the same cell if you want the computations to be done all at once.

In [6]:
d = 8
b = 2

#total number of polynomials is (2b+1)^(d/2). I want to spend an hour on each degree.

In [7]:
# start_time = time.time()

# polys = boxedPolys(12, 1)
# print(polys)
# polyData = [getStats(poly) for poly in polys]
# df = pd.DataFrame(polyData)
# df.columns = headers

# end_time = time.time()
# sec = end_time - start_time
# print(sec)
# #total number of polynomials is (2b+1)^(d/2). I want to spend an hour on each degree. 
# #Here, we processed 3^(d/2) polynomials So, I'm happy to process 3^(d/2)/sec*3600 polynomials.

In [12]:
def runParallel(d, num_cores = 24):
    start_time = time.time()

    polys = boxedPolys(d, 1)
    polyData = [getStats(poly) for poly in polys]
    df = pd.DataFrame(polyData)
    df.columns = headers

    end_time = time.time()
    sec = end_time - start_time
    
#     print(sec)
    #total number of polynomials is (2b+1)^(d/2). I want to spend an hour on each degree. 
    #Here, we processed 3^(d/2) polynomials So, I'm happy to process 3^(d/2)/sec*3600 polynomials.
    #So, I want to find b so that (2b+1)^(d/2) = 3^(d/2)/sec*3600. That is, 
    #(2b+1)=3*(3600/sec)^(2/d)
    #So, b = (3*(3600/sec)^(2/d)-1)/2
    b = floor((3*(3600*num_cores/sec)^(2/d)-1)/2)
    print(b)
    polys = boxedPolysParallel(d, b)
    with Pool() as pool: 
        polyData = pool.map(getStats, polys)
    df = pd.DataFrame(polyData)
    df.columns = headers
    df.to_csv(f'degree {d} up to {b}')
def run(d):
    start_time = time.time()

    polys = boxedPolys(d, 1)
    polyData = [getStats(poly) for poly in polys]
    df = pd.DataFrame(polyData)
    df.columns = headers2

    end_time = time.time()
    sec = end_time - start_time
    
#     print(sec)
    #total number of polynomials is (2b+1)^(d/2). I want to spend an hour on each degree. 
    #Here, we processed 3^(d/2) polynomials So, I'm happy to process 3^(d/2)/sec*3600 polynomials.
    #So, I want to find b so that (2b+1)^(d/2) = 3^(d/2)/sec*3600. That is, 
    #(2b+1)=3*(3600/sec)^(2/d)
    #So, b = (3*(3600/sec)^(2/d)-1)/2
    b = floor((3*(3600/sec)^(2/d)-1)/2)
    print(b)
#     polys = boxedPolysParallel(d, b)
#     with Pool() as pool: 
#         polyData = pool.map(getStats, polys)
    polys = http://localhost:8888/notebooks/Mahler_Stuff/Mahler-Measures-and-Galois-Groups-of-Reciprocal-Polynomials/Galois%20Groups/Galois%20Computations%20for%20Server.ipynb#boxedPolys(d, b)
    polyData = [getStats(poly) for poly in polys]
    df = pd.DataFrame(polyData)
    df.columns = headers
    df.to_csv(f'degree {d} up to {b}')
    

In [13]:
run(12)
run(14)
run(16)
run(18)
run(20)


3


KeyboardInterrupt: 